# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información



El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.



Seguirás los siguientes pasos:

Integrantes: Vickiann Jimenez, Gabriela Salazar, Jostin Vega

Descripción del Ejercicio



1. Proporcionar un Conjunto de Datos:

    * Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.

    * Consultas: Define un conjunto de consultas específicas.

    * Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.

In [33]:
import xml.etree.ElementTree as ET
import numpy as np

In [34]:
# Paso 1: Proporcionar un Conjunto de Datos
# Leer y parsear el archivo XML para obtener el corpus de documentos
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [35]:
# Función para procesar el texto y extraer palabras clave, útil para manejar el corpus y las consultas
def process_text(text):
    text = text.lower()
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    tokens = text.strip().split()
    return set(tokens)

In [36]:
# Funciones para calcular similitud Jaccard y Coseno
def jaccard_similarity(query, doc_keywords):
    intersection = len(query & doc_keywords)
    union = len(query | doc_keywords)
    return intersection / union if union else 0

In [37]:
def cosine_similarity(query, doc_keywords):
    intersection = len(query & doc_keywords)
    return intersection / (np.sqrt(len(query)) * np.sqrt(len(doc_keywords))) if len(doc_keywords) else 0


In [ ]:
# Leer el corpus de documentos
corpus = parse_corpus('03ranking_corpus.xml')


In [39]:
# Consultas: Definir un conjunto de consultas específicas
queries = {
    'query1': process_text("salud mental en estudiantes"),
    'query2': process_text("tecnología médica preventiva"),
}

In [40]:
# Juicios de Relevancia: Proporcionar una lista de qué documentos son relevantes para cada consulta
relevance_judgments = {
    'query1': {13, 14, 8, 7, 12},
    'query2': {1, 10, 20},
}

2. Calcular Resultados de Búsqueda:

    * Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.


In [41]:
# Generar resultados de búsqueda para cada sistema
def generate_results(system_type, queries, corpus):
    results = {}
    for query_id, query_keywords in queries.items():
        doc_scores = []
        for doc_id, doc_data in corpus.items():
            if system_type == 'jaccard':
                score = jaccard_similarity(query_keywords, doc_data['keywords'])
            elif system_type == 'cosine':
                score = cosine_similarity(query_keywords, doc_data['keywords'])
            doc_scores.append((doc_id, score))
        
        # Ordenar documentos por puntaje en orden descendente
        doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)
        # Tomar los IDs de los documentos en el orden de su puntaje
        results[query_id] = [doc_id for doc_id, _ in doc_scores[:5]]
    return results

In [42]:
# Resultados simulados utilizando los métodos Jaccard y Coseno
system_1_results = generate_results('jaccard', queries, corpus)
system_2_results = generate_results('cosine', queries, corpus)

In [43]:
system_1_results

{'query1': [2, 14, 13, 7, 11], 'query2': [10, 1, 20, 30, 2]}

In [44]:
system_2_results

{'query1': [2, 14, 13, 7, 11], 'query2': [10, 1, 20, 30, 2]}

3. Calcular las Métricas de Evaluación:

    * Calcular las siguientes métricas para cada sistema y consulta:

        * Precisión en el top-k (Prec@k)

        * Recall

        * F1-score

        * Mean Average Precision (MAP)

        * nDCG

In [45]:
# Función para calcular Precisión@K
def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    relevant_retrieved = [doc for doc in retrieved_k if doc in relevant]
    return len(relevant_retrieved) / k

In [46]:
# Función para calcular Recall
def recall(retrieved, relevant):
    relevant_retrieved = [doc for doc in retrieved if doc in relevant]
    return len(relevant_retrieved) / len(relevant) if relevant else 0

In [47]:
# Función para calcular F1-Score
def f1_score_custom(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

In [48]:
# Función para calcular Average Precision para una consulta específica
def average_precision(retrieved, relevant):
    relevant_retrieved = 0
    cumulative_precision = 0
    for i, doc_id in enumerate(retrieved, start=1):
        if doc_id in relevant:
            relevant_retrieved += 1
            cumulative_precision += relevant_retrieved / i
    return cumulative_precision / len(relevant) if relevant else 0

In [49]:
# Función para calcular Mean Average Precision (MAP)
def mean_average_precision(system_results, relevance_judgments):
    average_precisions = []
    for query, retrieved in system_results.items():
        relevant = relevance_judgments.get(query, set())
        average_precisions.append(average_precision(retrieved, relevant))
    return sum(average_precisions) / len(average_precisions)

In [50]:
# Función para calcular Discounted Cumulative Gain (DCG) hasta K
def dcg_at_k(retrieved, relevant, k):
    dcg = 0.0
    for i in range(min(k, len(retrieved))):
        if retrieved[i] in relevant:
            dcg += 1 / np.log2(i + 2)
    return dcg

In [51]:
# Función para calcular Normalized Discounted Cumulative Gain (nDCG) hasta K
def ndcg_at_k(retrieved, relevant, k):
    dcg = dcg_at_k(retrieved, relevant, k)
    ideal_dcg = dcg_at_k(sorted(relevant, key=lambda x: x in retrieved, reverse=True), relevant, k)
    return dcg / ideal_dcg if ideal_dcg > 0 else 0

4. Análisis y Comparación:

    * Comparar los resultados de los dos sistemas utilizando las métricas calculadas.

    * Discutir cuál sistema es más efectivo y por qué.

In [52]:
# Paso 4: Análisis y Comparación
for system_name, system_results in [('System 1 (Jaccard)', system_1_results), ('System 2 (Coseno)', system_2_results)]:
    print(f"\n--- Metrics for {system_name} ---")
    
    # Acumuladores de métricas para calcular el promedio al final
    total_prec_k = total_recall = total_f1 = total_map = total_ndcg = 0
    num_queries = len(queries)

    for query, relevant_set in relevance_judgments.items():
        results = system_results[query]
        
        # Calcular cada métrica para la consulta actual
        prec_k = precision_at_k(results, relevant_set, k=5)
        rec = recall(results, relevant_set)
        f1 = f1_score_custom(prec_k, rec)
        map_score = mean_average_precision({query: results}, {query: relevant_set})
        ndcg = ndcg_at_k(results, relevant_set, k=5)
        
        # Sumar las métricas para calcular promedios más adelante
        total_prec_k += prec_k
        total_recall += rec
        total_f1 += f1
        total_map += map_score
        total_ndcg += ndcg

        # Mostrar métricas por consulta para cada sistema
        print(f"\nResults for {query}:")
        print(f"  Prec@5: {prec_k:.4f}")
        print(f"  Recall: {rec:.4f}")
        print(f"  F1-Score: {f1:.4f}")
        print(f"  MAP: {map_score:.4f}")
        print(f"  nDCG@5: {ndcg:.4f}")

    # Calcular promedios de métricas para cada sistema
    avg_prec_k = total_prec_k / num_queries
    avg_recall = total_recall / num_queries
    avg_f1 = total_f1 / num_queries
    avg_map = total_map / num_queries
    avg_ndcg = total_ndcg / num_queries

    # Mostrar promedios finales para el sistema actual
    print(f"\nAverage Metrics for {system_name}:")
    print(f"  Avg Prec@5: {avg_prec_k:.4f}")
    print(f"  Avg Recall: {avg_recall:.4f}")
    print(f"  Avg F1-Score: {avg_f1:.4f}")
    print(f"  Avg MAP: {avg_map:.4f}")
    print(f"  Avg nDCG@5: {avg_ndcg:.4f}")


--- Metrics for System 1 (Jaccard) ---

Results for query1:
  Prec@5: 0.6000
  Recall: 0.6000
  F1-Score: 0.6000
  MAP: 0.3833
  nDCG@5: 0.5296

Results for query2:
  Prec@5: 0.6000
  Recall: 1.0000
  F1-Score: 0.7500
  MAP: 1.0000
  nDCG@5: 1.0000

Average Metrics for System 1 (Jaccard):
  Avg Prec@5: 0.6000
  Avg Recall: 0.8000
  Avg F1-Score: 0.6750
  Avg MAP: 0.6917
  Avg nDCG@5: 0.7648

--- Metrics for System 2 (Coseno) ---

Results for query1:
  Prec@5: 0.6000
  Recall: 0.6000
  F1-Score: 0.6000
  MAP: 0.3833
  nDCG@5: 0.5296

Results for query2:
  Prec@5: 0.6000
  Recall: 1.0000
  F1-Score: 0.7500
  MAP: 1.0000
  nDCG@5: 1.0000

Average Metrics for System 2 (Coseno):
  Avg Prec@5: 0.6000
  Avg Recall: 0.8000
  Avg F1-Score: 0.6750
  Avg MAP: 0.6917
  Avg nDCG@5: 0.7648
